# Init

In [200]:
# data_dir
DATA_DIR = "./data"

# create connection
library(RPostgres)
wrds <- dbConnect(Postgres(),
                  host='wrds-pgdata.wharton.upenn.edu',
                  port=9737,
                  dbname='wrds',
                  sslmode='require',
                  user='xiaomowu',
                  password='SLCyz2018')

# WRDS

## ind-profile

402,835 obs

Contains variables such as `gender`, `dob`, `nationality`...

In [207]:
library = 'boardex'
dataset = 'na_dir_profile_details' # individual profiles

res <- dbSendQuery(wrds, sprintf("select * from %s.%s", library, dataset))
individual_prof <- dbFetch(res, n=-1) %>% setDT()
dbClearResult(res)
sv(individual_prof, path='.')

-individual_prof- saved  (5.95 secs)


## ind-edu

1,341,622 obs

In [9]:
library = 'boardex'
dataset = 'na_dir_profile_education' # individual education

res <- dbSendQuery(wrds, sprintf("select * from %s.%s", library, dataset))
individual_edu <- dbFetch(res, n=-1) %>% setDT()
dbClearResult(res)
sv(individual_edu)

-individual_edu- saved  (6.68 secs)


## ind-emp

6,302,551 obs

Contains "which persons have served in the focal company"

In [12]:
library = 'boardex'
dataset = 'na_wrds_dir_profile_emp' # individual education

res <- dbSendQuery(wrds, sprintf("select * from %s.%s", library, dataset))
individual_emp <- dbFetch(res, n=-1) %>% setDT()
dbClearResult(res)
sv(individual_emp)

-individual_emp- saved  (47.63 secs)


## firm-profile

919,051 obs

Which contains `firm-id` and its `ticker`

In [2]:
library = 'boardex'
dataset = 'na_wrds_company_profile' # individual education

res <- dbSendQuery(wrds, sprintf("select * from %s.%s", library, dataset))
firm_prof_from_boardex <- dbFetch(res, n=-1) %>% setDT()
dbClearResult(res)
sv(firm_prof_from_boardex)

-firm_prof_from_boardex- saved  (6.06 secs)


In [ ]:
library = 'crspa'
dataset = 'msf' # individual education

res <- dbSendQuery(wrds, sprintf("select * from %s.%s", library, dataset))
firm_prof_from_crsp <- dbFetch(res, n=-1) %>% setDT()
dbClearResult(res)
sv(firm_prof_from_crsp)

## firm-net

3,631,160 obs

In [4]:
library = 'boardex'
dataset = 'na_wrds_company_networks' # individual education

res <- dbSendQuery(wrds, sprintf("select * from %s.%s", library, dataset))
firm_net <- dbFetch(res, n=-1) %>% setDT()
dbClearResult(res)
sv(firm_net)

-firm_net- saved  (38.16 secs)


## ind-net (dep)

Didn't use, because it's too large (806,646,080 obs) 

In [ ]:
library = 'boardex'
dataset = 'na_wrds_individual_networks' # individual education

res <- dbSendQuery(wrds, sprintf("select * from %s.%s", library, dataset))
individual_net <- dbFetch(res, n=-1) %>% setDT()
dbClearResult(res)
sv(individual_net)

# Clean

## ind_prof

Table `f_ind_prof`
- The profiles of the individuals that we may use. Each person has one exact observation. Note that we may not use all of them.
- For demo of `f_ind_prof` please see the end of this section
- `name`: the name of the individual
- `name_id`: unique id for that individual
- `dob`: date of birth
- `dod`: date of death
- `age`: the age given by the database. Seems to be calcualted by 2015.
- `gender`: the gender of the individual
- `nationality`: nationality of the individual

In [208]:
ld(individual_prof, force=T, path='.')

-individual_prof- loaded  (3.07 secs)


In [210]:
f_ind_prof = individual_prof[, 
    .(name=directorname, name_id=directorid, 
      dob=dob,
      dod=dod, 
      age=as.integer(age)+4, gender=gender,
      nationality=nationality)] %>%
    unique()
f_ind_prof[1]
sv(f_ind_prof, path='.')
# fwrite(f_ind_prof, './data/f_ind_prof.csv')

name,name_id,dob,dod,age,gender,nationality
David Shaw,16,22 Apr 1951,n.a.,72,M,American


-f_ind_prof- saved  (2.96 secs)


## ind_edu

Table `f_ind_edu`
- Education of the individuals that we may use. Each person may have MULTIPLE observations, if they receive education from multiple institutions. Note that we may not use all of them.
- For demo of `f_ind_edu` please see the end of this section
- `name`: the name of the individual
- `name_id`: unique id for that individual
- `edu_type`: type of education. Could be "Education" or "Professional Qualifications"
- `university`: the institution the individual receive education from. 
- `university_id`: the unique id for each `university`
- `qualification`: the type of qualificatoin, e.g. BA, PhD... 
- `award_date`: the date to be awarded the qualification.

In [6]:
ld(individual_edu, force=T)
f_ind_edu = individual_edu[,
    .(name=directorname, name_id=directorid, edu_type=rowtype,
      university=companyname, university_id=companyid,
      qualification=qualification, award_date=awarddate)] %>%
    unique()
f_ind_edu[1]
fwrite(f_ind_edu, './data/f_ind_edu.csv')

-individual_edu- loaded  (1.64 secs)


name,name_id,edu_type,university,university_id,qualification,award_date
David Shaw,16,Education,University of New Hampshire,62520,BA,1973-01-01


In [7]:
ld(f_ind_edu)

-f_ind_edu- loaded  (6.02 secs)


In [11]:
x = f_ind_edu[, table(university)] %>% as.data.table()
nrow(x)
x[order(-N)]

[1] 32440

university,N
American Institute of Certified Public Accountants (AICPA),44398
Unknown Organisation,20290
Harvard Business School,14617
Stanford University,12736
Wharton School University of Pennsylvania,12726
Harvard University,10956
University of Michigan,9955
University of California Berkeley,8730
Cornell University,8693
University of Texas at Austin,8024


## ind_emp

Table `f_ind_emp`
- Employment history of each individual. One individual may have MULTIPLE observations, if he/she has served in multiple companies.
- For demo of `f_ind_emp` please see the end of this section
- `name`: the name of the individual
- `name_id`: unique id for that individual
- `company`: the name of the company that the individual has served in.
- `company_id`: unique id for each `company`.
- `in_board`: "Yes" if the individual was in the "board of directors", otherwise "No".
- `in_supervisory`: "Yes" if the individual was in the "supervisory board", otherwise "No". 
- `role`: the role of the indiviudal when he/she was in the company
- `role_start_date`: the date when the individual started his/her role
- `role_end_date`: the date when the individual ended his/her role.

In [7]:
ld(individual_emp, force=T)

-individual_emp- loaded  (20.55 secs)


In [8]:
f_ind_emp = individual_emp[,
    .(name=directorname, name_id=directorid,
      company=companyname, company_id=companyid, 
      in_board=brdposition, in_supervisory=ned, role=rolename,
      role_start_date=datestartrole, role_end_date=dateendrole)] %>%
    unique()
fwrite(f_ind_emp, './data/f_ind_emp.csv')
# sv(f_ind_emp)
f_ind_emp[1]

name,name_id,company,company_id,in_board,in_supervisory,role,role_start_date,role_end_date
Ambassador Gerald Carmen,372547,Home Shopping Latino Inc (1 800 Autotow Inc prior to 11/2006),2,Yes,Yes,Director - SD,1998-07-01,NA


## firm_prof

Table `f_firm_prof`
- Profiles for the companies that we may use. Each company has exactly one observation
- For demo of `f_firm_prof` please see the end of this section
- `company`: the name of the company
- `company_id`: unique id for each `company`.
- `ticker`: ticker of the company, e.g. "MSFT" for Microsoft

In [177]:
ld(firm_prof_from_boardex, force=T, path='.')

-firm_prof_from_boardex- loaded  (5.79 secs)


In [181]:
firm_prof_from_boardex[str_detect(boardname, 'CONYERS')]

boardname,hoaddress1,hoaddress2,hoaddress3,hoaddress4,hoaddress5,hocountryname,hotelnumber,hofaxnumber,hourl,financialurl,companypolicy,ccaddress1,ccaddress2,ccaddress3,ccaddress4,ccaddress5,cccountryname,cctelnumber,ccfaxnumber,cikcode,sector,index,orgvisible,orgtype,boardid,clientcompanyid,ticker,isin,countryofquote,currency,revenuevaluedate,mktcapitalisation,noemployees,revenue,advisorname,advtypedesc
CONYERS PARK II ACQUISITION CORP,1 Greenwich Office Park,2nd Floor,Greenwich,Connecticut CT,06831,United States,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0001776661,Blank Check / Shell Companies,NA,Yes,Quoted,3059116,NA,CPAAU,US2128962030,U.S.A.,USD,NA,467,2,NA,Continental Stock Transfer & Trust Co LLC,Transfer Agents
CONYERS PARK II ACQUISITION CORP,1 Greenwich Office Park,2nd Floor,Greenwich,Connecticut CT,06831,United States,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0001776661,Blank Check / Shell Companies,NA,Yes,Quoted,3059116,NA,CPAAU,US2128962030,U.S.A.,USD,NA,467,2,NA,Kirkland & Ellis LLP (KE),Solicitors
CONYERS PARK II ACQUISITION CORP,1 Greenwich Office Park,2nd Floor,Greenwich,Connecticut CT,06831,United States,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0001776661,Blank Check / Shell Companies,NA,Yes,Quoted,3059116,NA,CPAAU,US2128962030,U.S.A.,USD,NA,467,2,NA,WithumSmith+Brown PC,Auditors


In [19]:
cols = c('hoaddress1', 'hoaddress2', 'hoaddress3', 'hoaddress4', 'hoaddress5', 'hocountryname', 'ccaddress1', 'ccaddress2', 'ccaddress3', 'ccaddress4', 'ccaddress5', 'cccountryname')

f_firm_prof = firm_prof_from_boardex[
    countryofquote=='U.S.A.' & hocountryname=='United States'
    ][, c(cols) := lapply(.SD, str_replace_na, ''), .SDcols=cols
    ][, .(company=boardname, company_id=boardid, sector,
      headoffice_address=str_c(
        hoaddress1, hoaddress2, hoaddress3,
        hoaddress4, hoaddress5, hocountryname, sep=' '),
      communication_address=str_c(
        ccaddress1, ccaddress2, ccaddress3,
        ccaddress4, ccaddress5, cccountryname, sep=' '),
      state=hoaddress4,
      market_cap=mktcapitalisation,
      n_employee=noemployees,
      revenue)
    ][state=='Alabama', state := 'Alabama AL'
    ][order(-market_cap)] %>% unique()
# fwrite(f_firm_prof, './data/f_firm_prof.csv')
sv(f_firm_prof, path='.')
f_firm_prof[1]

-f_firm_prof- saved  (0.1 secs)


company,company_id,sector,headoffice_address,communication_address,state,market_cap,n_employee,revenue
MICROSOFT CORP,20601,Software & Computer Services,One Microsoft Way Redmond Washington WA 98052-6399 United States,One Microsoft Way Redmond Washington WA 98052-6399 United States,Washington WA,1060781,131000,110360


In [20]:
f_firm_prof[order(-market_cap)][1:10]

company,company_id,sector,headoffice_address,communication_address,state,market_cap,n_employee,revenue
MICROSOFT CORP,20601,Software & Computer Services,One Microsoft Way Redmond Washington WA 98052-6399 United States,One Microsoft Way Redmond Washington WA 98052-6399 United States,Washington WA,1060781,131000,110360
AMAZON.COM INC,1703,General Retailers,410 Terry Avenue North Seattle Washington WA 98109-5210 United States,410 Terry Avenue North Seattle Washington WA 98109-5210 United States,Washington WA,894229,647500,232887
APPLE INC (Apple Computer Inc prior to 01/2007),2355,Information Technology Hardware,One Apple Park Way Cupertino California CA 95014 United States,1 Infinite Loop Cupertino California CA 95014 United States,California CA,888113,132000,265595
ALPHABET INC (Google Inc prior to 10/2015),116351,Software & Computer Services,1600 Amphitheatre Parkway Mountain View California CA 94043 United States,1600 Amphitheatre Parkway Mountain View California CA 94043 United States,California CA,776176,98771,136819
BERKSHIRE HATHAWAY INC,4080,Insurance,3555 Farnam Street Omaha Nebraska NE 68131 United States,3555 Farnam Street Omaha 68131 United States,Nebraska NE,491060,389000,247837
FACEBOOK INC,1820060,Software & Computer Services,1601 Willow Road Menlo Park California CA 94025 United States,1601 Willow Road Menlo Park California CA 94025 United States,California CA,457557,35587,55838
JPMORGAN CHASE & CO,17528,Banks,383 Madison Avenue New York NY 10179-0001 United States,383 Madison Avenue 10179-0001 United States,New York NY,354223,256105,109029
JOHNSON & JOHNSON,17478,Pharmaceuticals and Biotechnology,One Johnson & Johnson Plaza New Brunswick New Jersey NJ 08933 United States,One Johnson & Johnson Plaza New Brunswick New Jersey NJ 08933 United States,New Jersey NJ,343247,135100,81581
WALMART INC (Wal-Mart Stores Inc prior to 02/2018),33046,General Retailers,702 Southwest 8th Street Bentonville Arkansas AR 72716-0215 United States,702 Southwest 8th Street Bentonville Arkansas AR 72716 United States,Arkansas AR,330613,2200000,514405
EXXON MOBIL CORP,11430,Oil & Gas,5959 Las Colinas Boulevard Irving Texas TX 75039-2298 United States,5959 Las Colinas Boulevard Irving Texas TX 75039-2298 United States,Texas TX,306311,71100,290212


In [21]:
f_firm_prof[, table(sector)] %>% as.data.table() 

sector,N
Aerospace & Defence,56
Automobiles & Parts,72
Banks,848
Beverages,28
Blank Check / Shell Companies,122
Business Services,374
Chemicals,173
Clothing & Personal Products,82
Construction & Building Materials,148
Consumer Services,41


## firm_net

Table `f_firm_net`
- Network between firms. Perhapse the most import table in our analysis.
- Each line stands for one link from the `from_company` to the `to_company`
- Definition of company link: if one person has served both in company A and company B (not necessarily at the same time), then we say a link has be established from A to B through the person.
- For demo of `f_firm_net` please see the end of this section
- `from_company`: the start company of the link.
- `from_company_id`: the unque id for the `from_company`
- `to_company`: the end company of the link.
- `to_company_id`: the unique id for the `to_company`. 
- `name`: the name of the individual that established the link
- `name_id`: unique id for `name`.
- `role_in_from_company`: the role of the individual when he/she is in the `from_company`.
- `role_in_from_company_start_date`: the date when the individual started his/her role in the `from_company` 
- `role_in_from_company_end_date`: the date when the individual ended his/her role in the `from_company` 
- `role_in_to_company`: the role of the individual when he/she is in the `to_company`.
- `role_in_to_company_start_date`: the date when the individual started his/her role in the `to_company` 
- `role_in_to_company_end_date`: the date when the individual ended his/her role in the `to_company`

In [11]:
ld(firm_net, force=T)
f_firm_net = firm_net[,
    .(from_company=boardname, from_company_id=boardid,
      to_company=companyname, to_company_id=companyid,
      name=directorname, name_id=directorid,
      role_in_from_company=role,
      role_in_from_company_start_date=startcompanydatestartrole,
      role_in_from_company_end_date=startcompanydateendrole,
      role_in_to_company=associatedrole,
      role_in_to_company_start_date=conncompanydatestartrole,
      role_in_to_company_end_date=conncompanydateendrole)] %>%
    unique()
fwrite(f_firm_net, './data/f_firm_net.csv')
# sv(f_firm_net)
f_firm_net[1]

-firm_net- loaded  (11.14 secs)


from_company,from_company_id,to_company,to_company_id,name,name_id,role_in_from_company,role_in_from_company_start_date,role_in_from_company_end_date,role_in_to_company,role_in_to_company_start_date,role_in_to_company_end_date
GREENHILL & CO INC,40757,1 800 CONTACTS INC (De-listed 09/2007),3,Steve Key,59796,Independent Director (Brd) (SD),2004-05-01,NA,Independent Director (Brd) (SD),2005-07-28,2007-09-07


# Develop

## f_ind_emp

In [148]:
processed_company_data = fread('processed_company_data.csv')
ld(f_ind_emp, path='.')

f_ind_emp = f_ind_emp[company_id %in% processed_company_data[, unique(company_id)]]

-f_ind_emp- already exists, will NOT load again!  (0 secs)


In [63]:
sv(f_ind_emp, path='.')
fwrite(f_ind_emp, 'f_ind_emp.csv')

-f_ind_emp- saved  (3.24 secs)


## f_ind_prof

In [234]:
ld(f_ind_prof, path='.', force=T)
f_ind_emp = fread('f_ind_emp.csv')
f_ind_prof = f_ind_prof[name_id %in% f_ind_emp[, unique(name_id)]]

-f_ind_prof- loaded  (0.34 secs)


In [235]:
sv(f_ind_prof, path='.')
fwrite(f_ind_prof, 'f_ind_prof.csv')

-f_ind_prof- saved  (0.87 secs)


## f_ind_edu

In [65]:
ld(f_ind_edu, path='.')
f_ind_emp = fread('f_ind_emp.csv')

degree_bachelor = regex('(BS)|(BA)|(BE)|(Bachelor)')
degree_master = regex('(MS)|(Master)|(MA)|(MD)')
degree_phd = regex('(PhD)|(JD)|(MD)')

f_ind_edu = f_ind_edu[, ':='(degree='A', is_mba=0)
    ][str_detect(qualification, degree_bachelor) , ':='(degree='Bachelor')
    ][str_detect(qualification, degree_master) , ':='(degree='Master')
    ][str_detect(qualification, degree_phd) , ':='(degree='PhD')
    ][str_detect(qualification, 'MBA') , ':='(is_mba=1)
    ][order(name_id, -degree)
    ][, .SD[1], keyby=.(name_id)
    ][degree=='A', degree := 'Others']

-f_ind_edu- already exists, will NOT load again!  (0 secs)


In [66]:
sv(f_ind_edu, path='.')
fwrite(f_ind_edu, 'f_ind_edu.csv')

-f_ind_edu- saved  (3.83 secs)


## f_firm_prof

In [231]:
ld(f_firm_prof, force=T, path='.')
ld(f_ind_emp, force=T, path='.')
ld(f_ind_prof, force=T, path='.')
processed_company_data = fread('processed_company_data.csv')

f_ind_emp = f_ind_emp[in_board %in% c('Yes', 'Inside')
    ][, ':='(role_start_date=ymd(role_start_date))
    ][order(name_id, company, role_start_date)
    ][, .SD[1], keyby=.(name_id, company_id)
    ][, .(name_id, company_id, role_start_year=year(role_start_date))]

-f_firm_prof- loaded  (0.01 secs)
-f_ind_emp- loaded  (1.15 secs)
-f_ind_prof- loaded  (0.41 secs)


In [227]:
x1 = f_ind_emp[processed_company_data, on=.(company_id), nomatch=0
    ][f_ind_edu[, .(name_id, is_mba, degree)], on=.(name_id), nomatch=0]

x2 = f_ind_emp[processed_company_data, on=.(company_id), nomatch=0]
x2 = f_ind_edu[, .(name_id, is_mba, degree)
    ][x2, on=.(name_id)]

x3 = f_ind_emp[processed_company_data, on=.(company_id), nomatch=0
    ][f_ind_edu[, .(name_id, is_mba, degree)], on=.(name_id), nomatch=0
    ][f_ind_prof[, .(name_id, yob=2019-age, gender, nationality)], on=.(name_id), nomatch=0]


nrow(x1)
nrow(x2)
nrow(x3)

[1] 39203

[1] 41826

[1] 39165

In [223]:
1-392/418

[1] 0.06220096

In [228]:
40/41826

[1] 0.0009563429

In [232]:
f_firm_prof = f_ind_emp[processed_company_data, on=.(company_id), nomatch=0
    ][f_ind_edu[, .(name_id, is_mba, degree)], on=.(name_id), nomatch=0
    ][f_ind_prof[, .(name_id, yob=2019-age, gender, nationality)], on=.(name_id), nomatch=0
    ][, ':='(age=role_start_year-yob)
    ][, .(company=company[1], sector=sector[1], market_cap=market_cap[1], revenue=revenue[1], 
          n_employee=n_employee[1], state=state[1], headoffice_address=headoffice_address[1], 
          lat=lat[1], lng=lng[1],
          mba_pct=sum(is_mba)/.N, 
          non_mba_pct=sum(!is_mba)/.N,
          degree_bachelor_pct=sum(degree=='Bachelor')/.N, 
          degree_master_pct=sum(degree=='Master')/.N,
          degree_phd_pct=sum(degree=='PhD')/.N,
          degree_others_pct=sum(!degree %in% c('Bachelor', 'Master', 'PhD'))/.N,
          male_pct=sum(gender=='M')/.N,
          female_pct=sum(gender=='F')/.N,
          us_pct=sum(nationality=='American', na.rm=T)/.N,
          non_us_pct=sum(nationality!='American', na.rm=T)/.N,
          na_nationality_pct=sum(is.na(nationality))/.N),
      keyby=.(company_id)]

In [233]:
sv(f_firm_prof, path='.')
fwrite(f_firm_prof, 'f_firm_prof.csv')

-f_firm_prof- saved  (0.04 secs)


## link: firm-uni

In [18]:
ld(f_ind_emp, path='.')
ld(f_firm_prof, path='.')
ld(f_ind_edu, path='.')

-f_ind_emp- already exists, will NOT load again!  (0 secs)
-f_firm_prof- already exists, will NOT load again!  (0 secs)
-f_ind_edu- loaded  (2.06 secs)


In [15]:
uni_coord = fread('tmp_university.csv')[, .(university_id=id, university_lat=lat, university_lng=lng)]
uni_coord[1]
sv(uni_coord, path='.')

-f_ind_emp- already exists, will NOT load again!  (0 secs)
-f_firm_prof- already exists, will NOT load again!  (0 secs)


university_id,university_lat,university_lng
62520,43.13895,-70.93703


-uni_coord- saved  (0 secs)


In [19]:
top1k_company_id = f_firm_prof[1:1000, company_id]

link_firm_uni = f_ind_emp[in_board=='Yes' & company_id %in% top1k_company_id,
    .(name_id, company, company_id)
    ][f_ind_edu[university_id != 0, 
                .(name_id, university, university_id)], 
      nomatch=0,
      on=.(name_id)
    ][, .(company=company[1], university=university[1],
          n_grad=uniqueN(name_id)), 
      keyby=.(company_id, university_id)
    ][f_firm_prof[, .(company_id, company_lat=lat, company_lng=lng)],
      on=.(company_id)
    ][uni_coord, nomatch=0, on=.(university_id)]

link_firm_uni[1]
sv(link_firm_uni, path='.')
fwrite(link_firm_uni, 'link_firm_uni.csv')

company_id,university_id,company,university,n_grad,company_lat,company_lng,university_lat,university_lng
14355,62520,L3HARRIS TECHNOLOGIES INC (Harris Corp prior to 07/2019),University of New Hampshire,1,28.09078,-80.63855,43.13895,-70.93703


-link_firm_uni- saved  (0.03 secs)


## uni_prof

In [44]:
ld(link_firm_uni)

-link_firm_uni- already exists, will NOT load again!  (0 secs)


In [20]:
top500_uni = link_firm_uni[, 
     .(university=university[1], n_grad=sum(n_grad)), 
     keyby=.(university_id)
    ][order(-n_grad)]
top500_uni[1]
sv(top500_uni, path='.')
fwrite(top500_uni, 'top500_uni.csv')

university_id,university,n_grad
63013,Harvard Business School,2140


-top500_uni- saved  (0 secs)


## link: firm-firm

We do two things here:

- remove company name
- only keep companies that exist in `processed_company_data.csv`

In [106]:
ld(f_firm_net, path='.', force=T)

-f_firm_net- loaded  (8.3 secs)


In [112]:
processed_company_id = fread('processed_company_data.csv')[, unique(company_id)]

f_firm_net = f_firm_net[from_company_id %in% processed_company_id & to_company_id %in% processed_company_id
    ][, ':='(from_company, to_company)]

fwrite(f_firm_net, 'f_firm_net.csv')

In [107]:
f_firm_net[1]

from_company,from_company_id,to_company,to_company_id,name,name_id,role_in_from_company,role_in_from_company_start_date,role_in_from_company_end_date,role_in_to_company,role_in_to_company_start_date,role_in_to_company_end_date
GREENHILL & CO INC,40757,1 800 CONTACTS INC (De-listed 09/2007),3,Steve Key,59796,Independent Director (Brd) (SD),2004-05-01,NA,Independent Director (Brd) (SD),2005-07-28,2007-09-07


## link: sector-sector

In [101]:
ld(f_firm_prof, path='.', force=T)
ld(f_firm_net, path='.', force=T)
company_id_pool = f_firm_prof[, unique(company_id)]

-f_firm_prof- loaded  (0.03 secs)
-f_firm_net- loaded  (8.96 secs)


In [102]:
f_firm_net_1 = f_firm_net[role_in_from_company_end_date<=role_in_to_company_start_date
    ][, .(from_company_id=from_company_id,
          to_company_id=to_company_id)]

f_firm_net_2 = f_firm_net[role_in_from_company_start_date>=role_in_to_company_end_date
    ][, .(from_company_id=to_company_id,
          to_company_id=from_company_id)]

f_ind_net = rbindlist(list(f_firm_net_1, f_firm_net_2), fill=T)[f_firm_prof[, .(company_id, sector)], 
      .(from_company_id, to_company_id, from_sector=sector),
      on=c('from_company_id==company_id'),
      nomatch=NULL
    ][f_firm_prof[, .(company_id, sector)],
      .(from_company_id, to_company_id, from_sector, to_sector=sector),
      on=c('to_company_id==company_id'),
      nomatch=NULL
    ][, .(n=.N), keyby=.(from_sector, to_sector)
    ][order(from_sector, -n),
      .SD[1:3],
      keyby=.(from_sector)]

f_ind_net_full = rbindlist(list(f_firm_net_1, f_firm_net_2), fill=T)[f_firm_prof[, .(company_id, sector)], 
      .(from_company_id, to_company_id, from_sector=sector),
      on=c('from_company_id==company_id'),
      nomatch=NULL
    ][f_firm_prof[, .(company_id, sector)],
      .(from_company_id, to_company_id, from_sector, to_sector=sector),
      on=c('to_company_id==company_id'),
      nomatch=NULL
    ][, .(n=.N), keyby=.(from_sector, to_sector)
    ][order(from_sector, -n)]

In [103]:
sectors = f_ind_net[, sort(unique(c(from_sector, to_sector)))]
length(sectors)

f_ind_net[, ':='(from_sector=match(from_sector, sectors),
                 to_sector=match(to_sector, sectors),
                 id=1:.N)]
f_ind_net_full[, ':='(from_sector=match(from_sector, sectors),
                 to_sector=match(to_sector, sectors),
                 id=1:.N)]
f_ind_net_full[1:10]          

[1] 44

from_sector,to_sector,n,id
1,15,66,1
1,1,56,2
1,7,54,3
1,16,51,4
1,37,32,5
1,6,21,6
1,12,20,7
1,21,20,8
1,3,18,9
1,23,16,10


In [104]:
ind_matrix = matrix(0, nrow=44, ncol=44)
for (i in 1:nrow(f_ind_net)) {
    rowid=f_ind_net[i, from_sector]
    colid=f_ind_net[i, to_sector]
    n=f_ind_net[i, n]
    ind_matrix[rowid, colid]=n
}

ind_matrix_full = matrix(0, nrow=44, ncol=44)
for (i in 1:nrow(f_ind_net_full)) {
    rowid=f_ind_net_full[i, from_sector]
    colid=f_ind_net_full[i, to_sector]
    n=f_ind_net_full[i, n]
    ind_matrix_full[rowid, colid]=n
}

ind_matrix_full
fwrite(ind_matrix, 'ind_matrix.csv', col.names=F)
fwrite(ind_matrix_full, 'ind_matrix_full.csv', col.names=F)

56,14,18,0,0,21,54,2,13,0,0,20,2,0,66,51,2,8,3,2,20,8,16,12,0,4,0,0,2,6,2,8,0,0,12,4,32,0,10,8,0,12,12,6
8,52,4,0,4,18,38,6,32,0,8,16,4,2,74,52,2,2,10,12,12,10,14,2,0,8,2,4,6,2,10,18,0,6,14,21,14,20,12,6,0,36,14,0
8,14,451,8,2,51,18,18,24,4,4,12,10,28,50,26,16,12,6,54,18,4,16,74,14,36,0,6,18,10,24,8,8,12,60,6,48,158,5,30,6,8,94,10
4,8,30,14,0,6,2,10,4,0,4,4,0,2,6,0,0,18,4,4,2,6,4,2,0,25,0,0,4,0,0,2,0,0,0,0,8,10,0,2,2,2,12,2
0,0,0,2,0,0,0,0,2,0,0,0,0,0,4,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0
18,4,28,2,6,160,16,12,30,6,2,6,6,12,98,42,8,22,6,26,44,40,50,56,10,34,4,6,12,4,18,46,2,20,22,6,132,46,0,32,0,10,40,4
26,6,28,0,0,24,188,4,52,2,34,10,2,16,68,57,0,28,20,16,16,14,18,12,2,1,0,0,2,25,23,130,0,23,6,10,34,22,24,20,0,18,34,4
0,0,22,14,0,6,2,44,12,4,4,2,0,0,32,6,0,32,10,89,24,20,30,8,12,49,8,0,18,0,6,4,0,2,8,0,18,10,4,6,0,10,8,4
4,16,14,0,2,32,40,0,47,10,2,4,2,2,36,58,12,20,6,9,2,12,20,6,0,32,4,0,2,6,29,4,0,6,24,10,20,32,6,10,0,18,32,0
0,2,0,0,0,4,0,0,10,2,0,2,0,2,0,4,2,8,0,8,6,0,2,0,0,8,2,0,0,0,0,0,0,2,2,0,0,12,0,2,0,8,0,0
2,8,2,1,0,4,14,2,10,4,6,0,0,0,44,4,0,0,6,4,2,6,0,8,0,8,0,0,0,2,2,4,0,0,10,4,2,6,4,2,0,0,0,2


x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
